# Validação Geográfica

Este notebook implementa a validação geográfica cruzada, que serve como "prova de altíssima confiança" quando múltiplas fontes (OSM, CNEFE) concordam nas coordenadas geográficas de um endereço.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import math

# Constantes
RAIO_TERRA_KM = 6371.0  # Raio da Terra em km
DISTANCIA_MAXIMA_METROS = 100  # Distância máxima em metros para considerar match geográfico

In [ ]:
# UDF para calcular distância Haversine entre duas coordenadas
def distancia_haversine(lat1, lon1, lat2, lon2):
    """
    Calcula distância em metros entre duas coordenadas usando fórmula de Haversine
    
    Args:
        lat1, lon1: Latitude e longitude do primeiro ponto
        lat2, lon2: Latitude e longitude do segundo ponto
    
    Returns:
        Distância em metros
    """
    if not all([lat1, lon1, lat2, lon2]):
        return None
    
    # Converter para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Diferenças
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distância em metros
    distancia_km = RAIO_TERRA_KM * c
    distancia_metros = distancia_km * 1000
    
    return distancia_metros

udf_distancia_haversine = udf(distancia_haversine, DoubleType())

In [ ]:
# Função para validar correspondência geográfica entre fontes
def validar_correspondencia_geografica(df_matches, df_camada_ouro):
    """
    Valida matches usando correspondência geográfica cruzada
    
    Se múltiplas fontes concordam nas coordenadas, aumenta significativamente
    a confiança do match, mesmo que haja pequenas diferenças na grafia
    
    Args:
        df_matches: DataFrame com matches do motor de correspondência
        df_camada_ouro: DataFrame com Camada Ouro (contém coordenadas de múltiplas fontes)
    
    Returns:
        DataFrame com validação geográfica aplicada
    """
    # Enriquecer matches com informações geográficas da Camada Ouro
    df_com_geo = df_matches.join(
        df_camada_ouro.select(
            col("uid"),
            col("latitude").alias("lat_ouro"),
            col("longitude").alias("lon_ouro"),
            col("num_fontes").alias("num_fontes_geo"),
            col("fontes").alias("fontes_geo")
        ),
        col("uid_camada_ouro") == col("uid"),
        "inner"
    )
    
    # Se o endereço de entrada tiver coordenadas, validar distância
    # (assumindo que df_matches pode ter coordenadas se disponíveis)
    df_validado = df_com_geo \
        .withColumn("validacao_geografica", 
            when(
                (col("num_fontes_geo") >= 2) & 
                (col("lat_ouro").isNotNull()) & 
                (col("lon_ouro").isNotNull()),
                "VALIDADO"  # Múltiplas fontes concordam geograficamente
            ).otherwise("NAO_VALIDADO")
        ) \
        .withColumn("score_confianca_final", 
            when(
                (col("validacao_geografica") == "VALIDADO") & 
                (col("num_fontes_geo") >= 3),
                1.0  # Máxima confiança: 3+ fontes concordam geograficamente
            ).when(
                (col("validacao_geografica") == "VALIDADO") & 
                (col("num_fontes_geo") == 2),
                0.95  # Alta confiança: 2 fontes concordam geograficamente
            ).otherwise(
                col("score_final")  # Manter score original se não validado geograficamente
            )
        ) \
        .withColumn("nivel_confianca_final", 
            when(col("score_confianca_final") >= 0.95, "ALTISSIMA")
            .when(col("score_confianca_final") >= 0.90, "ALTA")
            .when(col("score_confianca_final") >= 0.75, "MEDIA")
            .otherwise("BAIXA")
        )
    
    return df_validado

In [ ]:
# Função para validar distância entre coordenadas de diferentes fontes
def validar_consistencia_geografica_fontes(df_camada_ouro):
    """
    Valida se coordenadas de diferentes fontes para o mesmo endereço
    estão próximas (dentro de um raio aceitável)
    
    Args:
        df_camada_ouro: DataFrame com Camada Ouro
    
    Returns:
        DataFrame com flag de consistência geográfica
    """
    # Para endereços com múltiplas fontes, verificar se coordenadas estão próximas
    # Isso requer acesso aos dados brutos das fontes
    # Por enquanto, assumimos que a média já foi calculada na Camada Ouro
    
    df_com_consistencia = df_camada_ouro \
        .withColumn("tem_coordenadas", 
            when((col("latitude").isNotNull()) & (col("longitude").isNotNull()), True)
            .otherwise(False)
        ) \
        .withColumn("consistencia_geografica", 
            when(
                (col("num_fontes") >= 2) & 
                (col("tem_coordenadas") == True),
                "CONSISTENTE"  # Múltiplas fontes com coordenadas
            ).when(
                (col("num_fontes") == 1) & 
                (col("tem_coordenadas") == True),
                "UNICA_FONTE"  # Apenas uma fonte com coordenadas
            ).otherwise(
                "SEM_COORDENADAS"
            )
        )
    
    return df_com_consistencia

In [ ]:
# Função para calcular distância entre match e coordenadas da Camada Ouro
def calcular_distancia_match(df_matches_com_geo):
    """
    Calcula distância entre coordenadas do match (se disponíveis) e Camada Ouro
    
    Args:
        df_matches_com_geo: DataFrame com matches e informações geográficas
    
    Returns:
        DataFrame com distância calculada
    """
    # Se houver coordenadas no endereço de entrada, calcular distância
    df_com_distancia = df_matches_com_geo \
        .withColumn("distancia_metros", 
            when(
                (col("latitude").isNotNull()) & 
                (col("longitude").isNotNull()) &
                (col("lat_ouro").isNotNull()) & 
                (col("lon_ouro").isNotNull()),
                udf_distancia_haversine(
                    col("latitude"),
                    col("longitude"),
                    col("lat_ouro"),
                    col("lon_ouro")
                )
            ).otherwise(None)
        ) \
        .withColumn("distancia_ok", 
            when(
                (col("distancia_metros").isNotNull()) & 
                (col("distancia_metros") <= DISTANCIA_MAXIMA_METROS),
                True
            ).otherwise(False)
        )
    
    return df_com_distancia

In [ ]:
# Pipeline completo de validação geográfica
def executar_validacao_geografica_completa(df_matches):
    """
    Executa validação geográfica completa nos matches
    
    Args:
        df_matches: DataFrame com matches do motor de correspondência
    
    Returns:
        DataFrame com validação geográfica aplicada
    """
    print("1. Carregando Camada Ouro...")
    df_camada_ouro = read_delta_table(PATH_CAMADA_OURO)
    
    print("2. Validando correspondência geográfica...")
    df_validado = validar_correspondencia_geografica(df_matches, df_camada_ouro)
    
    print("3. Calculando distâncias...")
    df_com_distancia = calcular_distancia_match(df_validado)
    
    print("4. Aplicando boost de confiança para validação geográfica...")
    df_final = df_com_distancia \
        .withColumn("boost_geografico", 
            when(
                (col("validacao_geografica") == "VALIDADO") & 
                (col("num_fontes_geo") >= 2),
                0.1  # Boost de 10% para validação geográfica
            ).otherwise(0.0)
        ) \
        .withColumn("score_confianca_final", 
            least(1.0, col("score_confianca_final") + col("boost_geografico"))
        ) \
        .select(
            col("id_endereco_input"),
            col("endereco_livre"),
            col("endereco_normalizado"),
            col("uid_camada_ouro"),
            col("endereco_camada_ouro"),
            col("score_final").alias("score_similaridade"),
            col("score_confianca_final"),
            col("nivel_confianca_final"),
            col("validacao_geografica"),
            col("num_fontes_geo"),
            col("fontes_geo"),
            col("lat_ouro").alias("latitude"),
            col("lon_ouro").alias("longitude"),
            col("distancia_metros"),
            col("distancia_ok"),
            current_timestamp().alias("validado_em")
        )
    
    print(f"   Matches validados: {df_final.count()}")
    print(f"   Matches com validação geográfica: {df_final.filter(col('validacao_geografica') == 'VALIDADO').count()}")
    
    return df_final

In [ ]:
# Exemplo de uso
# df_matches = read_delta_table(PATH_MATCHES)
# df_validado = executar_validacao_geografica_completa(df_matches)

# Visualizar resultados
# df_validado.show(20, truncate=False)

# Estatísticas de validação geográfica
# df_validado.groupBy("validacao_geografica", "nivel_confianca_final").agg(
#     count("*").alias("quantidade")
# ).orderBy(desc("quantidade")).show()

# Matches com altíssima confiança (validação geográfica + múltiplas fontes)
# df_validado.filter(
#     (col("nivel_confianca_final") == "ALTISSIMA") & 
#     (col("validacao_geografica") == "VALIDADO")
# ).show(10, truncate=False)

In [ ]:
# Salvar matches validados
# save_delta_table(df_validado, PATH_MATCHES, mode="overwrite", partition_by=["nivel_confianca_final"])

print("Validação geográfica concluída!")